In [ ]:
import gymnasium as gym
import aero_gym
from aero_gym.tools import evaluate, plotfile
import numpy as np
import math
from gymnasium.utils.env_checker import check_env
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
t_max = 20
delta_t = 0.1
t = np.linspace(0, t_max, int(t_max/delta_t)+1)

In [ ]:
h_ddot_impulse = np.zeros(len(t))
h_ddot_impulse[0] = 1.0
h_ddot_ones = np.ones(len(t))
h_ddot_binary_periodic = (np.sin(t) > 0).astype(int) - 0.01*(np.sin(t) < 0).astype(int)

## Impulse

In [ ]:
env = gym.make(
    'aero_gym/wagner-v0', 
    t_max=10, 
    delta_t=delta_t, 
    h_ddot_prescribed=h_ddot_impulse,
    observe_alpha_eff=True,
    reward_type=3)

In [ ]:
env.observation_space

In [ ]:
env.reset()
env.step([0.0])

In [ ]:
check_env(env.unwrapped)

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

## Step

In [ ]:
env = gym.make(
    'aero_gym/wagner-v0', 
    render_mode="ansi", 
    t_max=t_max, 
    delta_t=delta_t, 
    h_ddot_prescribed=h_ddot_ones,
    reward_type=3,
    h_dot_termination=False,
    )

In [ ]:
obs, info, render_list = evaluate(env)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)

## Gaussian

In [ ]:
def gaussian(x, a, b, c):
    return a * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def dgaussian(x, a, b, c):
    return a * -(x - b) / c ** 2 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

def ddgaussian(x, a, b, c):
    return a * (x ** 2 + b ** 2 - 2 * b * x - c ** 2) / c ** 4 * math.exp(-(x - b) ** 2 / (2 * c ** 2))

In [ ]:
a = -2/3*0.1
b = 3.0
c = 0.4
h_ddot_dgaussian = [dgaussian(ti, a, b, c) for ti in t]

a = 0.015
b = 15.0
c = 0.4
alpha_ddot_ddgaussian = [ddgaussian(ti, a, b, c) for ti in t]

In [ ]:
env = gym.make(
    'aero_gym/wagner-v0', 
    render_mode="ansi", 
    t_max=t_max, 
    delta_t=delta_t, 
    h_ddot_prescribed=h_ddot_dgaussian,
    reward_type=3,
    observe_alpha_eff=True,
    alpha_dot_termination=False,
    alpha_termination=False,
    h_dot_termination=False)

In [ ]:
obs, info, render_list = evaluate(env, alpha_ddot_prescribed=alpha_ddot_ddgaussian)

In [ ]:
%config InlineBackend.figure_format = 'svg'
fig, axarr = plotfile(info)